# Packages/Imports

In [1]:
from PySide6.QtWidgets import QApplication, QWidget
from tinydb import TinyDB, Query
from tinydb.table import Document
import re
import random as rd
import math

# Helper Functions

In [3]:
# Takes in a player level or creature CR and outputs the corresponding proviciency modifier.
def profByLevel(level):
    return 2 + int((level - 1) / 4)

# Takes in an ability score and returns the associated bonus score
def getBonus(score):
    return math.floor((score - 10) / 2)

"""
rollDice takes in a string expression representing a series of dice and randomizes a total in that range
stmt: A string representing the dice expression
hasAvg: Boolean value determining if an expression has a predetermined average value
rollAvg: Boolean value representing if the average value was requested or not
Returns: An integer representing the value of the expression
"""
def rollDice(stmt, hasAvg=False, rollAvg=False):
    # Cleans the statement of any unexpected characters
    stmt = re.sub(r"[^d0-9+-:]", "", stmt.lower())
    total = 0  # Total number rolled

    print(stmt)

    # Special case for expressions that have an average value (Expressed before dice with a colon)
    if hasAvg:
        # Splits the average value (behind the colon) from the dice expression
        splitAvg = stmt.split(":")
        # Returns the average if requested
        if rollAvg:
            return int(splitAvg[0])
        # Otherwise, reassigns stmt to conform with the rest of the method
        stmt = splitAvg[1]

    # Splits the statement across additions of different dice values
    dice = stmt.split("+")
    for d in dice:
        # Checks for a dice value, if none, add the term directly
        if "d" in d:
            # Splits term into dice quantity and value
            die = d.split("d")
            # If no quantity of dice was given, assume 1
            if die[0] == "":
                die[0] = "1"
            for i in range(int(die[0])):
                total += rd.randint(1, int(die[1]))
        else:
            total += int(d)
    return total

# Global Data

In [4]:
encounterXP = 0

# List of all valid damage types in dnd 5e and 2024 edition
damageType = [
    "Piercing",
    "Bludgeoning",
    "Slashing",
    "Cold",
    "Fire",
    "Lightning",
    "Thunder",
    "Poison",
    "Acid",
    "Necrotic",
    "Radiant",
    "Force",
    "Psychic"
]

# List of all valid alignments in dnd 5e and 2024 edition
alignments = [
    "Unaligned",
    "Lawful Good",
    "Lawful Neutral",
    "Lawful Evil",
    "Neutral Good",
    "True Neutral",
    "Neutral Evil",
    "Chaotic Good",
    "Chaotic Neutral",
    "Chaotic Evil"
]

# Dictionary for translating stat names into list order
statDict = {
    'Strength' : 0,
    'Dexterity' : 1,
    'Constitution' : 2,
    'Intelligence' : 3,
    'Wisdom' : 4,
    'Charisma' : 5
}

# Database Files

In [8]:
players = TinyDB('TestDatabases/players.json').table('player_characters')
monsters = TinyDB('TestDatabases/monsters.json').table('monsters')
types = TinyDB('TestDatabases/type.json').table('monster_types')
classes = TinyDB('TestDatabases/class.json').table('player_classes')
species = TinyDB('TestDatabases/species.json').table('species')
conditions = TinyDB('TestDatabases/condition.json').table('conditions')
parties = TinyDB('TestDatabases/party.json').table('parties')
refs = TinyDB('TestDatabases/reference.json')
senses = refs.table('senses')
sizes = refs.table('sizes')
skills = refs.table('skills')

# Core Object Definitions

In [6]:
class UnexpectedSyntax:
    "Unexpected syntax in expression."
    pass

In [7]:
class Combatant:
    def __init__(self, source, init):
        # Permanent values assigned from the database entry
        self.index = source.doc_id
        self.name = source["name"]
        self.ac = source['ac']
        self.size = source['size']
        self.alignment = source['alignment']
        self.languages = source['languages']
        self.speed = source['speed']
        self.stats = source['ability_scores']
        self.saves = source['saves']
        self.skillProf = source['skills']
        self.senses = source['senses']
        self.damages = source['damage_types']
        self.notes = source['notes']
        # Temporary values assigned during combat
        self.init = init
        self.tempHP = 0
        self.conditions = []
        self.concentration = False
        self.conscious = True
        # Set HP stats to zero as a baseline (overridden later)
        self.maxHP = 0
        self.currentHP = 0
        self.proficiency = 1

    """
    setCurrentHP takes in an expression and adjusts a combatant's currentHP attribute to match
    val: String representing the expression
    """
    def setCurrentHP(self, val):
        # Trim all unexpected characters from the string
        val = re.sub(r"[^0-9+-]", '', val)
        # Check if user wants to add to the current hp (and ensures hp isn't over max)
        if val[0] == '+' and '+' not in val[1:]:
            self.currentHP += int(val[1:])
            if self.currentHP > self.maxHP: self.currentHP = self.maxHP
        # Check if user wants to subtract from current hp (and checks for unconsciousness)
        elif val[0] == '-' and '-' not in val[1:]:
            self.currentHP -= int(val[1:])
            if self.currentHP <= 0:
                self.currentHP = 0
                self.conscious = False
        # Check to see if the expression only contains digits
        elif '+' not in val and '-' not in val:
            self.currentHP = int(val)
        # Raises an exception if the expression contains extra characters
        else:
            raise UnexpectedSyntax
 
    """
    updateTempHP takes in an expression and adjusts a combatant's tempHP attribute to match
    val: String representing the expression
    """
    def updateTempHP(self, val):
        # Trim all unexpected characters from the string
        val = re.sub(r"[^0-9+-]", '', val)
        # Check if user wants to add to the current hp (and ensures hp isn't over max)
        if val[0] == '+' and '+' not in val[1:]:
            self.tempHP += int(val[1:])
        # Check if user wants to subtract from current hp (and checks for unconsciousness)
        elif val[0] == '-' and '-' not in val[1:]:
            self.tempHP -= int(val[1:])
            if self.tempHP <= 0:
                self.tempHP = 0
        # Check to see if the expression only contains digits
        elif '+' not in val and '-' not in val:
            self.tempHP = int(val)
        # Raises an exception if the expression contains extra characters
        else:
            raise UnexpectedSyntax

    # Rolls a saving throw for the given stat.
    def rollSave(self, stat):
        # Builds a statement for rolling a d20 + ability modifier
        stmt = "d20+" + str(getBonus(self.stats[statDict[stat]]))
        if self.saves[statDict[stat]] == 1:
            stmt += str(self.proficiency)
        return rollDice(stmt)

    # Rolls a skill check using the name of the skill.
    def rollSkill(self, skillName):
        # Looks for the requested skill in reference database
        lookup = Query()
        skill = skills.search(lookup.skill == skillName)[0]
        # Finds the proper list in the ability array
        stat = statDict[skill['stat']]
        # Builds the rolling statement for d20 + ability modifier
        stmt = "d20+" + str(getBonus(self.stats[stat]))
        match self.skillProf[skill.doc_id]:
            case 1:
                stmt += "+" + str(self.proficiency)
            case 2:
                stmt += "+" + str(self.proficiency * 2)
        return rollDice(stmt)

    # Updates the notes field for the given combatant.
    def updateNotes(self, txt):
        self.notes = txt
    
    # Updates the concentration field to represent if the combatant is concentration or not
    def updateConcentrate(self, con):
        self.concentration = con

    # Adds a condition to the combatant's list
    def addCondition(self, cond):
        self.conditions.append(cond)

    # Removes a condition from the combatant's list
    def removeCondition(self, cond):
        self.conditions.remove(cond)


class Player(Combatant):
    def __init__(self, pc, init):
        # Set all shared values through the super method
        super().__init__(pc, init)
        # Set values unique to a player character
        self.level = pc['level']
        self.playerClass = pc['class']
        self.species = pc['species']
        self.currentHP = self.maxHP
        self.deathSaves = [0, 0]
        # Override default values
        self.maxHP = pc['hp']
        self.currentHP = self.maxHP
        self.proficiency = profByLevel(self.level)

    # Handles death saves either by inserted value or by rolling dice
    def deathSave(self, val, roll=True):
        if roll:
            val = rollDice("d20")
        if val >= 10:
            self.deathSave[1] += 1
        else:
            self.deathSave[0] += 1

class Monster(Combatant):
    def __init__(self, monst, init, isAvg):
        # Set all shared values through the super method
        super().__init__(monst, init)
        # Set values unique to a monster
        self.cr = monst["cr"]
        self.xp = monst["xp"]
        self.type = monst["type"]
        self.actions = monst["actions"]
        self.traits = monst["special_traits"]
        self.legend = monst["legendary"]
        self.lAct = monst["legendary_actions"]
        self.lRes = monst["legendary_resistances"]
        self.lair = monst["lair_actions"]
        # Override default values
        self.maxHP = rollDice(monst["hp"], hasAvg=True, rollAvg=isAvg)
        self.currentHP = self.maxHP
        self.proficiency = profByLevel(self.cr)

    # Overrides default method to add xp to encounter when monster is defeated.
    def setCurrentHP(self, val):
        super().setCurrentHP(val)
        if self.conscious == False:
            encounterXP += self.xp

# Testing Cells

In [15]:
for c in types.all():
    print(f"{c.doc_id}: {c['type']}" )

1: Abberation
2: Beast
3: Celestial
4: Construct
5: Dragon
6: Elemental
7: Fey
8: Feind
9: Giant
10: Humanoid
11: Monstrosity
12: Ooze
13: Plant
14: Undead
15: Test
16: Test
17: Test


In [14]:
types.upsert(Document({"type": "Abberation"}, doc_id=1))

[1]

In [ ]:
pc = Player(players.all()[0], 17)

In [14]:
p = players.all()[0]
[*p['class']]

['9']

In [11]:
player = players.all()[0]
18 + player['ability_scores'][1] / 100

18.2

In [10]:
dmgList = ["(1d6 + 3)Piercing", "(4)Cold", "(6d6)Fire"]
for dmg in dmgList:
    print(dmg)
    # | [+-] |[0-9]+|[a-zA-Z]{3,20}
    print(re.findall(r"[0-9]+d[0-9]{1,2}| [+-] |[0-9]+|[a-zA-Z]{3,20}", dmg))

(1d6 + 3)Piercing
['1d6', ' + ', '3', 'Piercing']
(4)Cold
['4', 'Cold']
(6d6)Fire
['6d6', 'Fire']
